In [12]:
# -*- coding:utf-8 -*-
import webbrowser
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# gen url to jisho
def webInfo(kan):
    webstem = 'https://jisho.org/search/'
    suffix = ""
    k = str((kan).encode("utf-8"))
    kanji = k[2:-1]
    chars = "\\x"
    kanji = kanji.replace(chars, '%')
    webstring = webstem + kanji + suffix
    return webstring


# Load webpage content
def jlptGrade(webstring):
    r = requests.get(webstring)
    soup = bs(r.content, features="html.parser")
    try:
        n = soup.find("span", string=re.compile('(JLPT|jlpt)')).text
        grade = n.split(' ')[1]
    except:
        grade = 'NA'
    return grade

In [13]:
# get csv to df for preparation
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/word_dictionary.csv'
df = pd.read_csv(path)
kanjis = df['kanji']




,word index,english,hiragana,kanji,date_added,rank,sample,is_active
0,0,house,いえ,家,09/05/2020,1,1,0
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0
2,2,nervous,ふあん,不安,09/05/2020,2,1,0
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1
4,4,to lend,かす,貸す,09/05/2020,2,1,0
...,...,...,...,...,...,...,...,...
608,608,mosquito,か,蚊,17/02/2021,2,5,1
609,609,business meeting; conference,かいぎ,会議,17/02/2021,5,4,1
610,610,environment,かんきょう,環境,17/02/2021,5,5,1
611,611,factory,こうじょう,工場,17/02/2021,3,4,1


In [3]:
# scrape grades
grades = []
for i,kanji in enumerate(kanjis):
    if i % 50 ==0:
        print(i, kanji)
    webstring = webInfo(kanji)
    grade = jlptGrade(webstring)
    grades.append(grade)

print(grades)


0 家
50 緑
100 最近
150 後ろ
200 妹
250 病気
300 恋愛的
350 取る
400 男性
450 伝える
500 泣く
550 夕方
600 差し上げる
['N5', 'N4', 'N3', 'N5', 'N5', 'NA', 'N4', 'N4', 'NA', 'N3', 'N5', 'N3', 'N3', 'N3', 'N5', 'N5', 'NA', 'NA', 'N4', 'N3', 'N4', 'N5', 'N5', 'N4', 'N5', 'N3', 'N4', 'NA', 'N5', 'N5', 'N3', 'N3', 'N4', 'N4', 'N5', 'N5', 'N2', 'NA', 'N5', 'N5', 'N2', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N4', 'N5', 'N3', 'N4', 'N5', 'N4', 'NA', 'N3', 'N5', 'N5', 'N5', 'N3', 'N3', 'N5', 'N3', 'NA', 'N4', 'N2', 'N4', 'N3', 'NA', 'N5', 'N3', 'N4', 'NA', 'N4', 'N1', 'N5', 'N4', 'N5', 'N5', 'N2', 'N5', 'N5', 'N3', 'N5', 'N4', 'N3', 'N4', 'N2', 'N3', 'N2', 'N5', 'N1', 'N4', 'N3', 'NA', 'N4', 'N2', 'NA', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N3', 'N4', 'N4', 'N5', 'N5', 'N5', 'N5', 'N5', 'N3', 'N4', 'N5', 'N5', 'NA', 'N5', 'NA', 'N4', 'N4', 'N5', 'N5', 'N5', 'NA', 'N5', 'NA', 'NA', 'N5', 'N5', 'N5', 'N3', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5

In [4]:
df['grade'] = grades
df.grade.value_counts()

N5    242
N4    134
NA     91
N3     88
N2     32
N1     26
Name: grade, dtype: int64

In [5]:
# get streak info
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/streak.csv'
df_str = pd.read_csv(path)
df['streak'] = df_str.streak
df['level'] = df.streak.apply(lambda x: 'expert' if x > 7 else 'advanced' if x > 2 else 'beginner')

In [6]:
# get results info
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/results2.csv'
df_res = pd.read_csv(path)
fails = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[0]
successes = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[1]
df['total_attempts'] = (fails + successes)
df['total_successes'] = successes
df['total_fails'] = fails
df['success_ratio'] = df['total_successes']/df['total_attempts']
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,608,mosquito,か,蚊,17/02/2021,2,5,1,N2,1,beginner,4.0,2.0,2.0,0.500000
609,609,business meeting; conference,かいぎ,会議,17/02/2021,5,4,1,N4,1,beginner,3.0,1.0,2.0,0.333333
610,610,environment,かんきょう,環境,17/02/2021,5,5,1,N3,0,beginner,4.0,0.0,4.0,0.000000
611,611,factory,こうじょう,工場,17/02/2021,3,4,1,N4,1,beginner,4.0,2.0,2.0,0.500000


In [7]:
# age column
from datetime import datetime as dt

today_ = pd.to_datetime('today').normalize()

df['age'] = df.date_added.apply(lambda x: today_ - pd.to_datetime(dt.strptime(x,"%d/%m/%Y"))).dt.days
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000,287
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000,287
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000,287
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922,287
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333,287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,608,mosquito,か,蚊,17/02/2021,2,5,1,N2,1,beginner,4.0,2.0,2.0,0.500000,3
609,609,business meeting; conference,かいぎ,会議,17/02/2021,5,4,1,N4,1,beginner,3.0,1.0,2.0,0.333333,3
610,610,environment,かんきょう,環境,17/02/2021,5,5,1,N3,0,beginner,4.0,0.0,4.0,0.000000,3
611,611,factory,こうじょう,工場,17/02/2021,3,4,1,N4,1,beginner,4.0,2.0,2.0,0.500000,3


In [8]:
df.to_csv('kanji_data_cleansed.csv', index=False)

In [9]:
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000,287
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000,287
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000,287
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922,287
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333,287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,608,mosquito,か,蚊,17/02/2021,2,5,1,N2,1,beginner,4.0,2.0,2.0,0.500000,3
609,609,business meeting; conference,かいぎ,会議,17/02/2021,5,4,1,N4,1,beginner,3.0,1.0,2.0,0.333333,3
610,610,environment,かんきょう,環境,17/02/2021,5,5,1,N3,0,beginner,4.0,0.0,4.0,0.000000,3
611,611,factory,こうじょう,工場,17/02/2021,3,4,1,N4,1,beginner,4.0,2.0,2.0,0.500000,3


In [16]:
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/fact_streak.csv'
df2 = pd.read_csv(path)
df2

,index_word,streak,date,sample
0,1,8,04/08/2020,1
1,5,8,04/08/2020,1
2,16,8,04/08/2020,1
3,20,7,04/08/2020,1
4,39,8,04/08/2020,1
...,...,...,...,...
5617,608,1,19/02/2021,5
5618,609,1,19/02/2021,4
5619,610,0,19/02/2021,5
5620,611,1,19/02/2021,4
